In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
DRIVE_PATH = '/content/drive/MyDrive/DataScience[Практика]/Дататон[хакатон]/Task4'

In [ ]:
import pickle
with open(DRIVE_PATH + "/data/train/data_train.pickle", 'rb') as f:
  data_train = pickle.load(f)
with open(DRIVE_PATH + "/data/train/labels_train.pickle", 'rb') as f:
  labels_train = pickle.load(f)
with open(DRIVE_PATH + "/data/val/data_val.pickle", 'rb') as f:
  data_val = pickle.load(f)
with open(DRIVE_PATH + "/data/val/labels_val.pickle", 'rb') as f:
  labels_val = pickle.load(f)

###EVAL

In [97]:
import numpy as np
import cv2
from google.colab.patches import cv2_imshow

# если вы используте модель для детектирования добавьте ее здесь:
#################################################################


#################################################################

# Для достижения лучшего результата детектирования используйте
# исходный размер изображения
img_size = [200, 320, 3]

# параметры преобразования перспективы изображения


src = np.float32([[10, 200], [310, 200],
                  [235, 110], [85, 110]])

src_draw = np.array(src, dtype=np.int32)

dst = np.float32([[0, img_size[0]], [img_size[1], img_size[0]],
                  [img_size[1], 0], [0, 0]])

def detect_lane_center_points(frame):
    """ Детектирование точек центральной линии дорожной разметки
    Для каждого изображения необходимо получить массив из 200 (высота изображения)
    центральных точек, где каждый элемент массива - координата точки по оси х,
    а индекс этого элемента - координата точки по оси y (от 0 до 199)
    predicted_points = [120, 121, 121, 122, 123, 123 ... 147, 147, 148]

    Если вы работаете с изображением большего разрешения, приведите массив predicted_points
    к соответствующему виду (ровно 200 элементов, имеющих значение от 0 до 359)
    """


    #--------------------Warping----------------------#
    resized = cv2.resize(frame, (320, 200))

    r_channel = resized[:, :, 2]
    binary = np.zeros_like(resized)
    binary[(r_channel > 200)] = 1

    hls = cv2.cvtColor(resized, cv2.COLOR_BGR2HLS)
    s_channel = hls[:, :, 2]
    binary2 = np.zeros_like(hls)
    binary2[(s_channel > 160)] = 1

    allBinary = np.zeros_like(binary)
    allBinary[((binary == 1)|(binary2 == 1))] = 255

    M = cv2.getPerspectiveTransform(src, dst)
    warped = cv2.warpPerspective(allBinary, M, (img_size[1], img_size[0]), flags=cv2.INTER_LINEAR)
    #cv2_imshow(warped)



    #------------------Vertical Lines-------------------#
    histogram = np.sum(warped[warped.shape[0] // 2:, :], axis=0)

    midpoint = histogram.shape[0] // 2
    histogram = np.transpose(histogram)[0]

    WhitestColumnIndL = np.argmax(histogram[:midpoint])
    WhitestColumnIndR = np.argmax(histogram[midpoint:]) + midpoint

    warped_img_copy = warped.copy()
    cv2.line(warped_img_copy, (WhitestColumnIndL, 0), (WhitestColumnIndL, warped_img_copy.shape[0]), 200, 2)
    cv2.line(warped_img_copy, (WhitestColumnIndR, 0), (WhitestColumnIndR, warped_img_copy.shape[0]), 200, 2)

    warped_img = warped


    #---------------------Mean Line---------------------#
    nwindows = 8
    window_height = np.int(warped_img.shape[0] / nwindows)
    window_half_width = 25

    XCenterLeftWindow = WhitestColumnIndL
    XCenterRightWindow = WhitestColumnIndR

    left_lane_inds = np.array([], dtype=np.int16)
    right_lane_inds = np.array([], dtype=np.int16)

    box_visual = warped_img.copy()

    nonzero = warped_img.nonzero()
    WhitePixelIndY = np.array(nonzero[0])
    WhitePixelIndX = np.array(nonzero[1])

    for window in range(nwindows):
      win_y1 = warped_img.shape[0] - (window + 1) * window_height
      win_y2 = warped_img.shape[0] - window * window_height

      left_win_x1 = XCenterLeftWindow - window_half_width
      left_win_x2 = XCenterLeftWindow + window_half_width

      right_win_x1 = XCenterRightWindow - window_half_width
      right_win_x2 = XCenterRightWindow + window_half_width

      cv2.rectangle(box_visual, (left_win_x1, win_y1), (left_win_x2, win_y2), (50 + window * 21, 0, 0), 2)
      cv2.rectangle(box_visual, (right_win_x1, win_y1), (right_win_x2, win_y2), (0, 0, 50 + window * 21), 2)


      good_left_inds = ((WhitePixelIndY >= win_y1) & (WhitePixelIndY <= win_y2) & (WhitePixelIndX >= left_win_x1) & (WhitePixelIndX <= left_win_x2)).nonzero()[0]
      good_right_inds = ((WhitePixelIndY >= win_y1) & (WhitePixelIndY <= win_y2) & (WhitePixelIndX >= right_win_x1) & (WhitePixelIndX <= right_win_x2)).nonzero()[0]

      left_lane_inds = np.concatenate((left_lane_inds, good_left_inds))
      right_lane_inds = np.concatenate((right_lane_inds, good_right_inds))

      if len(good_left_inds) > 50:
        XCenterLeftWindow = np.int(np.mean(WhitePixelIndX[good_left_inds]))
        XCenterRightWindow = np.int(np.mean(WhitePixelIndX[good_right_inds]))


    box_visual[WhitePixelIndY[left_lane_inds], WhitePixelIndX[left_lane_inds]] = [255, 0, 0]
    box_visual[WhitePixelIndY[right_lane_inds], WhitePixelIndX[right_lane_inds]] = [0, 0, 255]
    #cv2_imshow(box_visual)

    leftx = WhitePixelIndX[left_lane_inds]
    lefty = WhitePixelIndY[left_lane_inds]
    rightx = WhitePixelIndX[right_lane_inds]
    righty = WhitePixelIndY[right_lane_inds]

    left_fit = np.polyfit(lefty, leftx, 2)
    right_fit = np.polyfit(righty, rightx, 2)

    center_fit = (left_fit + right_fit) / 2
    
    predicted_points = []

    for ver_ind in range(box_visual.shape[0]):
      gor_ind = center_fit[0] * ver_ind ** 2 + center_fit[1] * ver_ind + center_fit[2]
      #cv2.circle(box_visual, (int(gor_ind), int(ver_ind)), 2, (255, 0, 255), 1)

      predicted_points.append(int(gor_ind))
    #cv2_imshow(box_visual)

    return np.array(predicted_points)

###HELPERS

In [ ]:
# Helper functions
import cv2
import os
import numpy as np

# This function loads in images and their labels and places them in a list
# The list contains all images and their associated labels
# For example, after data is loaded, im_list[0][:] will be the first image-label pair in the list



def load_dataset(image_dir):
    # Populate this empty image list
    im_list = []


    images = os.listdir(image_dir+"/images")

    for image_name in images:

        frame = cv2.imread(image_dir+"/images/" + image_name)
        img_id = image_name.split('.')

        if not frame is None:
            # Append the image, and it's type ("none", "pedistrain", "no_drive","stop","way-out","no_entry","road_works","parking","a_unevenness") to the image list
            im_list.append((img_id[0], frame))


    return im_list

def load_points(im_list, image_dir):

    points_list = []

    for img in im_list:
        img_id = img[0]
        frame = img[1]

        file_points = open(image_dir + '/annotations/' + img_id + '.txt', "r")
        points = np.loadtxt(image_dir + '/annotations/' + img_id + '.txt', "int16")

        points_list.append([frame, img_id, points])

    return points_list


###MAIN

In [98]:
import math
import pickle

def load_data():
    """загрузка изображений датасета"""
    try:
        TRAIN_IMAGE_LIST = data_train
        TRAIN_POINTS_LIST = labels_train

        VAL_IMAGE_LIST = data_val
        VAL_POINTS_LIST = labels_val

        # with open(DRIVE_PATH + 'data_train.pickle', 'wb') as f:
        #   pickle.dump(TRAIN_IMAGE_LIST, f)
        
        # with open(DRIVE_PATH + 'points_train.pickle', 'wb') as f:
        #   pickle.dump(TRAIN_POINTS_LIST, f)

        # with open(DRIVE_PATH + 'data_val.pickle', 'wb') as f:
        #   pickle.dump(VAL_IMAGE_LIST, f)
        
        # with open(DRIVE_PATH + 'points_val.pickle', 'wb') as f:
        #   pickle.dump(VAL_POINTS_LIST, f)
    except Exception:
        TRAIN_POINTS_LIST = []
        VAL_POINTS_LIST = []

    return TRAIN_POINTS_LIST, VAL_POINTS_LIST

def predict(frame):
    predicted_points = detect_lane_center_points(frame)
    return predicted_points

def check_prediction(test_image_points, predicted_points):
    """ Проверка близости расположения предсказанной точки и реальной точки
    Если отклонение более чем на 10 % (difference > 18) точка считается
    не распознанной
    """
    check_predictioion_value = 0
    for point in test_image_points:
        x = point[0]
        y = point[1]


        if len(predicted_points.shape) == 1:
            predicted_points_x = predicted_points[y]
        else:
            predicted_points_x = predicted_points[y][0]
        difference = math.fabs(x - predicted_points_x)

        if difference > 18:
            check_predictioion_value += 1
        else:
            check_predictioion_value += difference / 180
    
    return check_predictioion_value/len(test_image_points)


def get_misclassified_images(test_images):
    """ Проверка точек для каждого изображения из набора"""
    count = 0.
    check_predictioion_value_frames = 0
    for frame in test_images[:3]:
        count += 1
        test_image_points = frame[2]
        predicted_points = predict(frame[0])
        check_predictioion_value_frames += check_prediction(test_image_points, predicted_points)
    error = check_predictioion_value_frames/len(test_images)
    return error




train_image_list, val_image_list = load_data()
# print(len(train_image_list), len(val_image_list))

MISCLASSIFIED = get_misclassified_images(train_image_list)

accuracy = 1 - MISCLASSIFIED
print(accuracy)

0.9973865030674847
